In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [17]:
df = pd.read_csv('./data/SQLiV3.csv', encoding='utf-8-sig')
df.head()

,Sentence,Label,Unnamed: 2,Unnamed: 3
0,""" or pg_sleep ( __TIME__ ) --",1,NaN,NaN
1,create user name identified by pass123 tempora...,NaN,1,NaN
2,AND 1 = utl_inaddr.get_host_address ( ...,1,NaN,NaN
3,select * from users where id = '1' or @ @1 ...,1,NaN,NaN
4,"select * from users where id = 1 or 1#"" ( ...",1,NaN,NaN


In [18]:
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3'])
df

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,NaN
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [19]:
df = df.dropna(subset=['Label'])
df

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [20]:
if df['Sentence'].isna().any():
    print("There are NaN values in the 'Sentence' column.")
else:
    print("There are no NaN values in the 'Sentence' column.")

There are NaN values in the 'Sentence' column.


In [21]:
df = df.dropna(subset=['Sentence'])
df

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [7]:
if df['Sentence'].isna().any():
    print("There are NaN values in the 'Sentence' column.")
else:
    print("There are no NaN values in the 'Sentence' column.")

There are no NaN values in the 'Sentence' column.


In [8]:
if df['Label'].isna().any():
    print("There are NaN values in the 'Sentence' column.")
else:
    print("There are no NaN values in the 'Sentence' column.")

There are no NaN values in the 'Sentence' column.


In [8]:
df[df['Label'].str.contains('Select*')]

,Sentence,Label
19897,SELECT * FROM Customers GO,EXEC SelectAllCustomers
19902,SELECT * FROM Products,/*Select all the columns of all the records i...
19936,SELECT * FROM [Brazil Customers],CREATE PROCEDURE SelectAllCustomers AS
19937,SELECT * FROM Customers GO,EXEC SelectAllCustomers


In [10]:
df = df[isinstance(df['Label'].iloc[0], int)]

KeyError: False

In [35]:
df.drop(df.index[df["Label"].apply(lambda x: not (x.strip().isnumeric()))], axis=0, inplace=True)

AttributeError: 'int' object has no attribute 'strip'

In [23]:
df

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [11]:
df[df['Label'].str.contains('Select*')]

,Sentence,Label


In [ ]:
df = df[df['Label'].isin([0, 1])]
df

In [24]:
df

,Sentence,Label
0,""" or pg_sleep ( __TIME__ ) --",1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
...,...,...
30914,DELETE FROM door WHERE grow = 'small',0
30915,DELETE FROM tomorrow,0
30916,SELECT wide ( s ) FROM west,0
30917,SELECT * FROM ( SELECT slide FROM breath ),0


In [13]:
df = df.drop(df[~df['Label'].isin([0, 1])].index)
df

,Sentence,Label


In [25]:
def data2char_index(X, max_len):
    """
    This function takes a list of strings and converts each string to a list of integers, where each integer represents
    the index of a character in the alphabet string. The alphabet string contains all the characters that are allowed
    in the input strings. The resulting list of integer lists is then padded with zeros to ensure that all lists have
    the same length, which is equal to the max_len parameter.

    Args:
    - X: list of strings
    - max_len: integer, maximum length of each list of integers after padding

    Returns:
    - X_char: numpy array of shape (len(X), max_len), where each element is an integer representing the index of a
              character in the alphabet string
    """
    alphabet = " abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    result = [] 
    for data in X:
        mat = []
        for ch in data:
            if ch not in alphabet:
                continue
            mat.append(alphabet.index(ch))
        result.append(mat)   
    X_char = tf.keras.preprocessing.sequence.pad_sequences(np.array(result, dtype=object), padding='post',
                                                           truncating='post', maxlen=max_len)
    return X_char

In [26]:
data = df['Sentence'].values
label = df['Label'].values

trainX, testX, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

x_train = data2char_index(trainX, max_len=1000)
x_test = data2char_index(testX, max_len=1000)

In [27]:
x_train.shape

(24487, 1000)

In [28]:
x_test.shape

(6122, 1000)

In [36]:
def get_charcnn_model(max_len):
    """
    This function returns a compiled Keras model for character-level convolutional neural network (CNN) for XSS classification.
    The model takes as input a tensor of shape (max_len,), where max_len is the maximum length of the input strings.
    The model architecture consists of three parallel 1D convolutional layers with different filter sizes (5, 10, and 15),
    followed by max pooling layers, and a concatenation layer that combines the outputs of the three parallel layers.
    The concatenated output is then flattened and passed through two fully connected (Dense) layers with ReLU activation,
    followed by a final output layer with sigmoid activation.

    Args:
    - max_len: integer, maximum length of the input strings

    Returns:
    - model: compiled Keras model
    """
    main_input = tf.keras.layers.Input(shape=(max_len,))
    
    embedder = tf.keras.layers.Embedding(
        input_dim=70,  
        output_dim=80, 
        input_length=max_len,
        trainable=False
    )
    embed = embedder(main_input)
    
    cnn1 = tf.keras.layers.Conv1D(32, 5, padding='same', strides=1, activation='relu')(embed)
    cnn1 = tf.keras.layers.MaxPooling1D(pool_size=12)(cnn1)
    
    cnn2 = tf.keras.layers.Conv1D(32, 10, padding='same', strides=1, activation='relu')(embed)
    cnn2 = tf.keras.layers.MaxPooling1D(pool_size=11)(cnn2)
    
    cnn3 = tf.keras.layers.Conv1D(32, 15, padding='same', strides=1, activation='relu')(embed)
    cnn3 = tf.keras.layers.MaxPooling1D(pool_size=10)(cnn3)
    
    cnn = tf.keras.layers.concatenate([cnn1, cnn2, cnn3], axis=1)
    flat = tf.keras.layers.Flatten()(cnn)
    drop = tf.keras.layers.Dropout(0.2)(flat)
    dense1 = tf.keras.layers.Dense(1024, activation='relu')(drop)
    dense2 = tf.keras.layers.Dense(128, activation='relu')(dense1)
    main_output = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
    model = tf.keras.Model(inputs=main_input, outputs=main_output)
    return model

In [37]:
model = get_charcnn_model(max_len=1000)
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1000, 80)     5600        ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 1000, 32)     12832       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 1000, 32)     25632       ['embedding_1[0][0]']            
                                                                                            

In [31]:
print(df['Label'])

0        1
2        1
3        1
4        1
5        1
        ..
30914    0
30915    0
30916    0
30917    0
30918    0
Name: Label, Length: 30609, dtype: object


In [33]:
df['Label'] = df['Label'].astype(int)

C:\Users\mohit\AppData\Local\Temp\ipykernel_38552\3577165278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].astype(int)


In [38]:
batch_size = 128
num_epoch = 5
model_log = model.fit(
    x_train, 
    y_train,
    batch_size=batch_size,
    epochs=num_epoch,
    verbose=1,
    validation_data=(x_test, y_test),
)

Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\mohit\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyn

In [31]:
pred = model.predict(x_test)
y_pred = np.int64(pred>0.5)

192/192 [==============================] - 6s 32ms/step


In [32]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9962430578242405 
 Precision : 0.9960681520314548 
 Recall : 0.993897122929381
